# [E09] Movie Recommendation
  
### 목차
1. 데이터 준비 및 전처리
2. 데이터 분석
3. 선호하는 영화 추가
4. CSR matrix 생성
5. 모델 설계 및 훈련
6. 훈련된 모델이 예측한 나의 선호도
7. 내가 좋아하는 영화와 비슷한 영화 추천
8. 내가 좋아할 만한 영화 추천
9. 회고

   
   ---
   
### 루브릭 평가 기준
1. CSR matrix가 정상적으로 만들어졌다.
    * 사용자와 아이템 갯수를 바탕으로 정확한 사이즈로 만들었다.
2. MF 모델이 정상적으로 훈련되어 그럴듯한 추천이 이루어졌다.
    * 사용자와 아이템 백터 내적 수치가 의미있게 형성되었다.
3. 비슷한 영화 찾기와 유저에게 추천하기의 과정이 정상적으로 진행되었다.
    * MF 모델이 예측한 유저 선호도 및 아이템간 유사도, 기여도가 의미있게 측정되었다.

## 1. 데이터 준비 및 전처리

In [1]:
import pandas as pd
import os

rating_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python', encoding = "ISO-8859-1")
orginal_data_size = len(ratings)
ratings.head()

,user_id,movie_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


* Movielens 데이터의 내용 불러오기
* ratings.dat 파일에는 user_id, movie_id, rating, timestamp column들이 존재

In [2]:
ratings = ratings[ratings['rating']>=3]
filtered_data_size = len(ratings)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


* 평점이 3점 이상인 movie들만 남김
* 1,000,209개의 영화 중, 836,478개의 영화만 평점이 3점 이상 (약 83.63%)

In [3]:
ratings.rename(columns={'rating':'count'}, inplace=True)
ratings['count']

0          5
1          3
2          3
3          4
4          5
          ..
1000203    3
1000205    5
1000206    5
1000207    4
1000208    4
Name: count, Length: 836478, dtype: int64

* rating column의 이름을 count로 변경

In [4]:
movie_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python', encoding='ISO-8859-1')
movies.head()

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


* movie_id에 해당하는 영화가 무엇인지 확인하기 위해서, movies.dat 파일을 불러옴

In [5]:
ratings = ratings.join(movies.set_index('movie_id'), on='movie_id')
ratings.head()

,user_id,movie_id,count,timestamp,title,genre
0,1,1193,5,978300760,One Flew Over the Cuckoo's Nest (1975),Drama
1,1,661,3,978302109,James and the Giant Peach (1996),Animation|Children's|Musical
2,1,914,3,978301968,My Fair Lady (1964),Musical|Romance
3,1,3408,4,978300275,Erin Brockovich (2000),Drama
4,1,2355,5,978824291,"Bug's Life, A (1998)",Animation|Children's|Comedy


* ratings dataframe과 movies dataframe 합치기

In [6]:
ratings = ratings.drop(columns=['timestamp', 'genre'])
ratings.head()

,user_id,movie_id,count,title
0,1,1193,5,One Flew Over the Cuckoo's Nest (1975)
1,1,661,3,James and the Giant Peach (1996)
2,1,914,3,My Fair Lady (1964)
3,1,3408,4,Erin Brockovich (2000)
4,1,2355,5,"Bug's Life, A (1998)"


* timestamp, gerne column은 불필요하므로, 삭제

## 2. 데이터 분석

In [7]:
print('# of movie_id: ', ratings['movie_id'].nunique())
print('# of title   : ', ratings['title'].nunique())
print('# of user_id : ', ratings['user_id'].nunique())

# of movie_id:  3628
# of title   :  3628
# of user_id :  6039


* ratings에 존재하는 유니크한 영화 갯수, 영화 제목 수, 사용자 수를 출력

In [8]:
movie_count = ratings.groupby('title')['user_id'].count()
movie_count.sort_values(ascending=False).head(30)

title
American Beauty (1999)                                   3211
Star Wars: Episode IV - A New Hope (1977)                2910
Star Wars: Episode V - The Empire Strikes Back (1980)    2885
Star Wars: Episode VI - Return of the Jedi (1983)        2716
Saving Private Ryan (1998)                               2561
Terminator 2: Judgment Day (1991)                        2509
Silence of the Lambs, The (1991)                         2498
Raiders of the Lost Ark (1981)                           2473
Back to the Future (1985)                                2460
Matrix, The (1999)                                       2434
Jurassic Park (1993)                                     2413
Sixth Sense, The (1999)                                  2385
Fargo (1996)                                             2371
Braveheart (1995)                                        2314
Men in Black (1997)                                      2297
Schindler's List (1993)                                  2257
Pr

* 사용자들이 많이 본 영화 30개 출력
* 가장 인기 있는 영화로 판단

In [9]:
user_count = ratings.groupby('user_id')['movie_id'].count()
user_count.describe()

count    6039.000000
mean      138.512668
std       156.241599
min         1.000000
25%        38.000000
50%        81.000000
75%       177.000000
max      1968.000000
Name: movie_id, dtype: float64

* 유저 별 몇 편의 영화를 봤는지 통계 확인

## 3. 선호하는 영화 추가

In [10]:
movies[movies['title'].str.lower().str.contains('men in', regex=False)]

,movie_id,title,genre
516,520,Robin Hood: Men in Tights (1993),Comedy
1539,1580,Men in Black (1997),Action|Adventure|Comedy|Sci-Fi


* 영화 제목을 키워드로 검색

In [11]:
my_favorite_id = [1240, 2571, 1580 , 648, 3623]

my_favorite_title = []
for i in my_favorite_id:
    my_favorite_title.extend(list(movies[movies['movie_id'] == i]['title']))

my_movielist = pd.DataFrame({'user_id': ['lil_park']*5, 'movie_id': my_favorite_id, 'count': [5]*5, 'title': my_favorite_title})
my_movielist

,user_id,movie_id,count,title
0,lil_park,1240,5,"Terminator, The (1984)"
1,lil_park,2571,5,"Matrix, The (1999)"
2,lil_park,1580,5,Men in Black (1997)
3,lil_park,648,5,Mission: Impossible (1996)
4,lil_park,3623,5,Mission: Impossible 2 (2000)


* 내가 좋아하는 영화 5편을 선정하여 id와 title 리스트 만들기
* user_id = lil_park, 각각의 영화를 5번씩 봤다고 가정

In [12]:
if not ratings.isin({'user_id':['lil_park']})['user_id'].any():
    ratings = ratings.append(my_movielist, ignore_index=True)

ratings.tail(10)

,user_id,movie_id,count,title
836473,6040,1090,3,Platoon (1986)
836474,6040,1094,5,"Crying Game, The (1992)"
836475,6040,562,5,Welcome to the Dollhouse (1995)
836476,6040,1096,4,Sophie's Choice (1982)
836477,6040,1097,4,E.T. the Extra-Terrestrial (1982)
836478,lil_park,1240,5,"Terminator, The (1984)"
836479,lil_park,2571,5,"Matrix, The (1999)"
836480,lil_park,1580,5,Men in Black (1997)
836481,lil_park,648,5,Mission: Impossible (1996)
836482,lil_park,3623,5,Mission: Impossible 2 (2000)


* 기존의 ratings dataframe에 새로운 데이터를 추가 (my_movielist)

## 4. CSR matrix 생성

In [13]:
user_unique = ratings['user_id'].unique()
movie_unique = ratings['title'].unique()

user_to_idx = {v:k for k,v in enumerate(user_unique)}
movie_to_idx = {v:k for k,v in enumerate(movie_unique)}

* user_unique: 유저의 수
* movie_unique: 영화의 갯수
  
  
* 유저 id와 영화 제목을 고유한 갯수만큼 indexing

In [14]:
temp_user_data = ratings['user_id'].map(user_to_idx.get).dropna()
if len(temp_user_data) == len(ratings):
    print('user_id column indexing OK!!')
    ratings['user_id'] = temp_user_data
else:
    print('user_id column indexing Fail!!')

temp_movie_data = ratings['title'].map(movie_to_idx.get).dropna()
if len(temp_movie_data) == len(ratings):
    print('title column indexing OK!!')
    ratings['title'] = temp_movie_data
else:
    print('title column indexing Fail!!')

ratings

user_id column indexing OK!!
title column indexing OK!!


,user_id,movie_id,count,title
0,0,1193,5,0
1,0,661,3,1
2,0,914,3,2
3,0,3408,4,3
4,0,2355,5,4
...,...,...,...,...
836478,6039,1240,5,200
836479,6039,2571,5,124
836480,6039,1580,5,175
836481,6039,648,5,58


* data column 값을 indexing 값으로 교체
* user_to_idx.get을 통해 user_id 컬럼의 모든 값을 인덱싱
* movie_to_idx을 통해 title 컬럼도 동일한 방식으로 인덱싱

In [15]:
from scipy.sparse import csr_matrix

num_user = ratings['user_id'].nunique()
num_movie = ratings['title'].nunique()

csr_data = csr_matrix((ratings['count'], (ratings.user_id, ratings.title)), shape= (num_user, num_movie))
csr_data

<6040x3628 sparse matrix of type '<class 'numpy.longlong'>'
	with 836483 stored elements in Compressed Sparse Row format>

* CSR matrix 생성

## 5. 모델 설계 및 훈련

In [16]:
from implicit.als import AlternatingLeastSquares
import os
import numpy as np

os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

* implicit 라이브러리에서 권장하고 있는 부분
* 의미에 대해서는 잘 모르겠음

In [17]:
als_model = AlternatingLeastSquares(factors=100, regularization=0.01, use_gpu=False, iterations=15, dtype=np.float32)

* Implicit AlternatingLeastSquares model 선언

In [18]:
csr_data_transpose = csr_data.T
csr_data_transpose

<3628x6040 sparse matrix of type '<class 'numpy.longlong'>'
	with 836483 stored elements in Compressed Sparse Column format>

* als model은 input으로 item X user 꼴의 matrix를 받기 때문에 Transpose가 필요 (행렬곱 연산)

In [19]:
als_model.fit(csr_data_transpose)

  0%|          | 0/15 [00:00<?, ?it/s]

* model 훈련

## 6. 훈련된 모델이 예측한 나의 선호도

In [20]:
lil_park, matrix = user_to_idx['lil_park'], movie_to_idx['Matrix, The (1999)']
lil_park_vector, matrix_vector = als_model.user_factors[lil_park], als_model.item_factors[matrix]

* model을 이용하여 나의 벡터와 영화 벡터를 구함
* 나의 벡터는 미리 추가해놓은 'lil_park' 데이터를 이용
* 내가 선호하는 영화 리스트에 포함되어 있는 영화인 Matrix의 벡터를 계산

In [21]:
lil_park_vector

array([ 4.10019904e-01, -3.80544811e-01, -3.47956493e-02, -6.23874702e-02,
        5.24455011e-01,  2.44025022e-01,  1.01598847e+00, -6.46223128e-01,
        1.10658455e+00,  7.77535915e-01, -9.55325812e-02,  5.51060557e-01,
        1.35940289e+00, -1.12588331e-01, -6.02340698e-01,  5.56486547e-01,
        4.24802601e-01,  3.90608818e-03, -6.13577127e-01, -8.04037929e-01,
       -1.21868916e-01, -4.97529268e-01,  3.78615260e-02,  6.24389239e-02,
       -2.57686436e-01,  1.14417136e+00, -6.34511590e-01, -6.32092595e-01,
       -7.33129442e-01, -1.44379082e-04, -4.32181180e-01,  9.42485332e-01,
       -4.55087423e-01, -2.34932527e-01,  2.18313292e-01,  3.50444764e-01,
        1.25276208e-01, -1.35127380e-01,  9.98782098e-01,  1.14024210e+00,
        2.17955559e-01,  4.84859020e-01,  9.02650535e-01, -1.29193902e-01,
        1.38803095e-01, -2.67581761e-01, -7.35978425e-01, -4.65282649e-01,
       -2.64441669e-01, -1.13362052e-01, -5.44064306e-02, -6.30889595e-01,
       -1.06813319e-01, -

* 나의 벡터

In [22]:
matrix_vector

array([ 0.00904326, -0.00272089, -0.0197151 ,  0.00329583,  0.01248024,
       -0.00185465,  0.02115068, -0.00436876,  0.02263998,  0.02628555,
        0.00432578,  0.01237044,  0.03842386,  0.02006912, -0.01634437,
        0.01149884,  0.01996523, -0.00385619,  0.00299031, -0.01101883,
       -0.00489077,  0.00348486,  0.01557175,  0.0030802 ,  0.00505475,
        0.03184101, -0.00857368,  0.01146542, -0.00769847,  0.01098374,
        0.01344242,  0.03095594, -0.00554795, -0.00495961,  0.03103339,
        0.02690692, -0.00646813,  0.00325101,  0.03801562,  0.02796322,
        0.0172271 ,  0.0466028 ,  0.01012817,  0.00253165, -0.01267212,
        0.01039502, -0.02360802, -0.00091741,  0.01080921,  0.02637553,
        0.01289073, -0.00673841, -0.01843985,  0.00070057, -0.01020727,
        0.00683116, -0.01981277, -0.01622631,  0.02238117,  0.02738811,
        0.0159368 ,  0.03235731,  0.00670442, -0.00888341,  0.00310398,
        0.00018562,  0.02150644, -0.01083803,  0.02058482,  0.01

* 영화 Matrix의 벡터

In [23]:
np.dot(lil_park_vector, matrix_vector)

0.62820446

* 나의 벡터와 영화 Matrix의 벡터를 내적
* 내가 선호하는 영화이므로, 내적 값이 큰 것을 확인할 수 있다.

In [24]:
Platoon = movie_to_idx['Platoon (1986)']
Platoon_vector = als_model.item_factors[Platoon]
np.dot(lil_park_vector, Platoon_vector)

-0.0666814

* 선호 리스트에 없는 영화의 벡터를 계산
* 나의 벡터와 선호 리스트에 없는 영화 벡터를 내적한 결과
* 굉장히 낮은 값이 출력

## 7. 내가 좋아하는 영화와 비슷한 영화 추천

In [25]:
favorite_movie = 'Mission: Impossible (1996)'
movie_id = movie_to_idx[favorite_movie]
similar_movie = als_model.similar_items(movie_id, N=15)
similar_movie 

[(58, 0.9999999),
 (138, 0.54618484),
 (118, 0.45856228),
 (129, 0.4476144),
 (82, 0.44286004),
 (772, 0.4405187),
 (766, 0.43479803),
 (587, 0.43452403),
 (156, 0.43022692),
 (119, 0.428624),
 (179, 0.4231625),
 (975, 0.41819593),
 (714, 0.4110838),
 (107, 0.39991465),
 (140, 0.38783678)]

* 내가 좋아하는 영화 벡터를 이용하여, input으로 주어진 영화와 비슷한 영화를 총 15개 출력

In [26]:
idx_to_movie = {v:k for k,v in movie_to_idx.items()}
[idx_to_movie[i[0]] for i in similar_movie]

['Mission: Impossible (1996)',
 'True Lies (1994)',
 'Con Air (1997)',
 'Conspiracy Theory (1997)',
 'Lost World: Jurassic Park, The (1997)',
 'Rising Sun (1993)',
 'GoldenEye (1995)',
 'Batman Returns (1992)',
 'Mask of Zorro, The (1998)',
 'Twister (1996)',
 'Rock, The (1996)',
 'Young Sherlock Holmes (1985)',
 'Waterworld (1995)',
 'Jurassic Park (1993)',
 'Clear and Present Danger (1994)']

* 앞서 구한 비슷한 영화 15개는 index로 출력이 되어있기 때문에,
* index 값을 이용하여 영화의 이름을 출력

In [27]:
def get_similar_movie(movie_title: str):
    movie_id = movie_to_idx[movie_title]
    similar_movie = als_model.similar_items(movie_id, N=15)
    similar_movie = [idx_to_movie[i[0]] for i in similar_movie]
    return similar_movie

* 앞서 작성한 코드인, 내가 선호하는 영화와 비슷한 영화를 추천해주는 함수
* 함수 내부에서 index 값을 이용하여 input으로 주어진 영화와 비슷한 영화의 제목을 출력

In [28]:
get_similar_movie('Men in Black (1997)')

['Men in Black (1997)',
 'Jurassic Park (1993)',
 'Terminator 2: Judgment Day (1991)',
 'Total Recall (1990)',
 'Independence Day (ID4) (1996)',
 'Fifth Element, The (1997)',
 'Matrix, The (1999)',
 'Lost World: Jurassic Park, The (1997)',
 'Face/Off (1997)',
 'True Lies (1994)',
 'Galaxy Quest (1999)',
 'Schlafes Bruder (Brother of Sleep) (1995)',
 'Rocky Horror Picture Show, The (1975)',
 'Grumpy Old Men (1993)',
 'Repo Man (1984)']

In [29]:
get_similar_movie('Terminator, The (1984)')

['Terminator, The (1984)',
 'Aliens (1986)',
 'Die Hard (1988)',
 'Alien (1979)',
 'Predator (1987)',
 'Terminator 2: Judgment Day (1991)',
 'Matrix, The (1999)',
 'Total Recall (1990)',
 'Blade Runner (1982)',
 'Robocop (1987)',
 'Star Wars: Episode V - The Empire Strikes Back (1980)',
 'Indiana Jones and the Last Crusade (1989)',
 'Mad Max 2 (a.k.a. The Road Warrior) (1981)',
 'Star Wars: Episode IV - A New Hope (1977)',
 'Raiders of the Lost Ark (1981)']

## 8. 내가 좋아할 만한 영화 추천

In [30]:
user = user_to_idx['lil_park']

movie_recommended = als_model.recommend(user, csr_data, N=15, filter_already_liked_items=True)
movie_recommended

[(92, 0.65678895),
 (107, 0.5143002),
 (62, 0.458377),
 (372, 0.3984903),
 (67, 0.3971084),
 (44, 0.3374477),
 (117, 0.29833728),
 (375, 0.2933408),
 (145, 0.29011288),
 (82, 0.28131947),
 (141, 0.2730414),
 (64, 0.2718894),
 (60, 0.261981),
 (193, 0.25484398),
 (346, 0.24959241)]

* 나의 벡터를 이용하여, 내가 선호하는 영화들과 비슷한 영화들을 index 값으로 반환

In [31]:
[idx_to_movie[i[0]] for i in movie_recommended]

['Terminator 2: Judgment Day (1991)',
 'Jurassic Park (1993)',
 'Total Recall (1990)',
 'X-Men (2000)',
 'Gladiator (2000)',
 'Star Wars: Episode IV - A New Hope (1977)',
 'Star Wars: Episode V - The Empire Strikes Back (1980)',
 'Face/Off (1997)',
 'Fifth Element, The (1997)',
 'Lost World: Jurassic Park, The (1997)',
 'Fugitive, The (1993)',
 'Star Wars: Episode VI - Return of the Jedi (1983)',
 'Star Wars: Episode I - The Phantom Menace (1999)',
 'Alien (1979)',
 'Patriot, The (2000)']

* 앞서 구한 index 값을 이용하여, 내가 선호할만한 영화 제목을 출력

In [32]:
recommended = movie_to_idx['Jurassic Park (1993)']
explain = als_model.explain(user, csr_data, itemid=recommended)

[(idx_to_movie[i[0]], i[1]) for i in explain[1]]

[('Men in Black (1997)', 0.31793220793943433),
 ('Matrix, The (1999)', 0.12040903555502976),
 ('Mission: Impossible (1996)', 0.0922334400200717),
 ('Terminator, The (1984)', 0.010533496308269667),
 ('Mission: Impossible 2 (2000)', -0.0350992890793501)]

* 사용자의 선호 영화 리스트에 있는 영화들이 영화 추천에 어느정도 기여했는지 기여도를 확인
* 'Jurassic Park (1993)'에 대한 기여도를 확인한 결과, 'Men in Black (1997)'이 0.3179 정도로 꽤 높은 기여를 함

## 9. 회고

* data의 경우, 평점과 제목이 2개의 파일로 나눠져 있었다. 평점과 영화 제목의 경우, 많이 사용하는 데이터이기 때문에, join을 이용하여 두 개의 dataframe을 하나로 합쳐서 사용하였다.
* 결과를 보면, 선호 영화 리스트에 있는 'Matrix, The (1999)'의 벡터 내적 값은 약 0.63으로, 나름 높은 값을 얻었고, 'Platoon (1986)'이라는 영화와의 벡터 내적 값은 약 -0.07정도로 굉장히 낮은 값을 얻었다. 대체적으로 액션, 블록버스터 영화를 선호하는 입장에서 'Platoon (1986)'이라는 영화를 선호하지 않는 나의 선호도가 나름 잘 반영되었다고 생각된다. 
* 내가 좋아하는 영화와 비슷한 추천 영화들에 대해서 살펴보면, 대체적으로 액션, 블록버스터와 같은 비슷한 장르의 영화들이 추천되었다. 또한, 내가 재밌게 본 영화나 좋아할만한 영화들이 추천된 것을 보면, 해당 장르에 대해서는 어느정도 추천 시스템 결과에 신뢰할 수 있다고 생각된다.
   
---
   
* 유튜브, 넷플릭스, 인터넷 광고 등 여러 플랫폼에서 사용하는 추천 시스템에 대해서 궁금한 부분이 많았다. 인터넷 광고는 사용자의 검색어를 기반으로 추천한다고 느꼈지만, 유튜브나 넷플릭스는 어떤 식으로 영상을 추천하는 것인지 이해하기 힘들었다. 이번 노드를 통해서 추천 시스템에 대한 어느정도의 궁금증을 해결한 것 같다. 그러나, 노드를 진행하면서 생긴 의문은, '과연 이 모델을 혹은 이 결과를 신뢰할 수 있는가?' 이다. 선호하는 것이 명확한 사용자도 있지만, 매칭이 잘 되지 않는 선호도를 가진 사용자도 있을 것이다. (가령, 애니메이션 영화를 좋아하면서 블록버스터 영화를 좋아하는 사용자) 혹은, 흔히 말하는 매니아처럼 하나의 장르만을 선호하는 사람이 아닌, 모든 장르를 선호하는 사람도 존재할 것이다. 그런 사람들에게 추천 시스템이 잘 적용될까? 영화의 내용을 전부 확인하고 추천하는 것이 아니기에 추천 시스템에는 한계가 존재하는 것일까? 확실히 이번 노드는 신기했지만, 어려웠고, 여러 의문이 생기는 노드였다.